In [1]:
using ModelingToolkit, OrdinaryDiffEq, Plots
using ModelingToolkit: t_nounits as t, D_nounits as d

### Models

In [2]:
@connector Pin begin
    v(t) # Potential at the pin
    i(t), [connect = Flow] # Current flowing into the pin
end

@mtkmodel OnePort begin
    @components begin
        p = Pin()
        n = Pin()
    end
    @variables begin
        v(t) # Voltage drop of the two pins (= p.v - n.v)
        i(t) # Current flowing into the pin
    end
    @equations begin
        v ~ p.v - n.v
        0 ~ p.i + n.i
        i ~ p.i
    end
end

In [3]:
@connector HeatPort begin
    T(t)
    Q_flow(t), [connect = Flow]
end

@connector HeatPort_a begin
    @extend T, Q_flow = heatPort_a = HeatPort()
end

@connector HeatPort_b begin
    @extend T, Q_flow = heatPort_b = HeatPort()
end

In [4]:
@mtkmodel FixedTemperature begin
    @components begin
        port = HeatPort_b()
    end
    @parameters begin
        T, [description = "Fixed temperature boundary condition"]
    end
    @equations begin
        port.T ~ T
    end
end

In [5]:
# copy from ModelingToolkitStandardLibrary
@mtkmodel HeatingResistor begin
    @extend v, i = oneport = OnePort()
    @components begin
        heat_port = HeatPort()
    end
    @parameters begin
        R_ref = 1.0, [description = "Reference resistance"]
        T_ref = 300.15, [description = "Reference temperature"]
        alpha = 0, [description = "Temperature coefficient of resistance"]
        tau = 1e-6, [description = "Time constant for resistance change"]
    end
    @variables begin
        R(t), [guess = R_ref]
    end
    @equations begin
        d(R) ~ (R_ref * (1 + alpha * (heat_port.T - T_ref)) - R) / tau
        heat_port.Q_flow ~ -v * i # -LossPower
        v ~ i * R 
        # ExtraVariablesSystemException: The system is unbalanced. 
        # There are 25 highest order derivative variables and 24 equations.
    end
end

In [6]:
@mtkmodel Ground begin
    @components begin
        p = Pin()
    end
    @equations begin
        p.v ~ 0
    end
end

@mtkmodel Capacitor begin
    @extend i,v = oneport = OnePort()
    @parameters begin
        C # Capacitance
    end
    @equations begin
        i ~ C * d(v)
    end
end

@mtkmodel ConstantVoltage begin
    @extend v, = oneport = OnePort()    
    @parameters begin
        V # Value of constant voltage
    end
    @equations begin
        v ~ V
    end
end

In [7]:
@mtkmodel RcHeating begin    
    @components begin
        resistor = HeatingResistor()           
        capacitor = Capacitor()
        source = ConstantVoltage()
        ground = Ground()
        fixedTemperature = FixedTemperature()
        
    end      
    @equations begin        
        connect(source.p, resistor.p)
        connect(resistor.n, capacitor.p)
        connect(capacitor.n, source.n, ground.p) 
        connect(resistor.heat_port, fixedTemperature.port)       
    end
end

### 0. sys

In [8]:
@mtkbuild sys = RcHeating()

In [9]:
"""
Unknowns (2)
    capacitor₊v(t) ▪ Unassigned ▪ Voltage drop of the two pins (= p.v - n.v)
    resistor₊i(t) ▪ Unassigned ▪ Current flowing from pin p to pin n
Parameters (6)
    resistor₊alpha ▪ 0 ▪ Temperature coefficient of resistance
    fixedTemperature₊T ▪ Unassigned ▪ Fixed temperature at port
    source₊V ▪ Unassigned ▪ Value of constant voltage
    resistor₊R_ref ▪ 1.0 ▪ Reference resistance
    resistor₊T_ref ▪ 300.15 ▪ Reference temperature
    capacitor₊C ▪ Unassigned ▪ Capacitance
Equations (2)
    Differential(t)(capacitor₊v(t)) ~ capacitor₊i(t) / capacitor₊C
    0 ~ -resistor₊v(t) + resistor₊i(t)*resistor₊R(t)
"""
nothing

In [10]:
parameters(sys) |> display
observed(sys) |> display
unknowns(sys) |> display
equations(sys) |> display

### .1 (sys)

In [11]:
function display_isys(isys)
    # isys_ps = parameters(isys)
    isys_eqs = equations(isys)
    isys_unks = unknowns(isys)
    isys_obs = observed(isys)

    # display("$(length(isys_ps)) parameters")
    # display(isys_ps)
    display("$(length(isys_eqs)) equations")
    display(isys_eqs)
    display("$(length(isys_unks)) unknowns")
    display(isys_unks)
    display("$(length(isys_obs)) observed")
    display(isys_obs)
end

In [12]:
isys1 = generate_initializesystem(sys)
display_isys(isys1)

In [13]:
isys1_prob = ODEProblem(sys, [], (0.0, 10.0))
# Warning: Did not converge after `maxiters = 100` substitutions

### .2 (sys; u0map, pmap)

In [14]:
isys2 = generate_initializesystem(sys; 
            u0map = [sys.capacitor.v => 0.0, sys.resistor.i => 0.0],
            pmap = [sys.capacitor.C => 1.0, sys.source.V=>1.0,sys.fixedTemperature.T=>400.15])
display_isys(isys2)

In [22]:
isys2_prob = ODEProblem(sys, [sys.capacitor.v => 0.0, sys.resistor.i => 0.0], (0.0, 10.0),
                [sys.capacitor.C => 1.0, sys.source.V=>1.0,sys.fixedTemperature.T=>400.15])

In [23]:
isys2_prob_initsys = isys2_prob.f.initializeprob.f.sys

In [25]:
isys2_sol = solve(isys2_prob, FBDF(); saveat=0.1)
plot(isys2_sol, idxs=[sys.capacitor.v, sys.resistor.i]) |> display

In [17]:
structural_simplify(isys2; fully_determined = false)

### .3 (sys; u0manp, pmap, guesses)

In [18]:
isys3 = generate_initializesystem(sys; 
            u0map = [sys.capacitor.v => 0.0, sys.resistor.i => 0.0],
            pmap = [sys.capacitor.C => 1.0, sys.source.V=>1.0,sys.fixedTemperature.T=>400.15],
            guesses = [sys.resistor.R=>sys.resistor.R_ref])
display_isys(isys3)

In [19]:
isys3_prob = ODEProblem(sys, [sys.capacitor.v => 0.0, sys.resistor.i => 0.0], (0.0, 10.0),
                [sys.capacitor.C => 1.0, sys.source.V=>1.0,sys.fixedTemperature.T=>400.15],
                guesses = [sys.resistor.R=>sys.resistor.R_ref])

In [20]:
isys3_prob_initsys = isys3_prob.f.initializeprob.f.sys

In [21]:
structural_simplify(isys3; fully_determined = false)